# Preprocesamiento de los comentarios para la primera fase de fine-tuning y entrenamiento del modelo
En esta primera fase, una vez extraídos los bodies de todos los issues/PRs de los repositorios listados junto con 2 comentarios de 250 del total de issues de cada repositorio, se prepararán todos los datos para ser utilizados en el entrenamiento y ajuste fino del modelo que se utilizará.

Se seguirán prácticamente los mismos pasos vistos en *c_preparing_data_for_statistics_and_ML* pero con varias diferencias claves que existen entre los modelos BERT que se utilizarán ahora y los modelos de clasificación presentados con anterioridad (notebooks de GVTIA).

Para Transformers funciona mejor un preprocesado mínimo y dejar la segmentación al propio tokenizador del modelo, a continuación se muestra que procedimientos similares a los anteriores se mantendrán y cuáles se evitarán:

## Se mantendrá:
- Normalización de espacios/saltos de línea
- Eliminación de caracteres de control raros o poco usuales
- Se conservará el uso de mayúsculas y minúsculas, signos, números, URLs, nombres propios de vulnerabilidades o bugs (CVE-XXXX-YYYY), rutas (/etc/...), código entre backticks (`return salida`), nombres de APIs.
- Se definirá una longitud máxima de tokens por comentario o el uso de *sliding window* si el texto es muy largo.

## Se omitirá:
- Pasar todo el texto a minúsculas, los modelos RoBERTa/DistilRoBERTa que se utilizarán utilizan mayúsculas y minúsculas.
- Eliminar la puntuación y stopwords.
- Stemming / lematización.
- Normalizaciones agresivas de URLs/código -> se pierde señal técnica.

Una vez explicado esto, se comenzará con el preprocesado de todos los comentarios extraídos de GitHub, comenzando como se ha visto ya en diversas ocasiones, con cargar el documento (.csv) en un dataframe de pandas para su uso y manipulación.

En este caso, se cuenta con 2 documentos:
- **gh_bodys_lastyear.csv**. Archivo que contiene los bodies (comentario principal) de todos los Issues/PRs en el último año de los repositorios listados para la extracción de comentarios.
- **gh_comments_lastyear.csv**. Archivo que contiene los 2 primeros comentarios de cada Issue/PR de 250 Issues/PRs por repositorio (500 comentarios por repo), en gran parte de los casos serán las respuestas aportadas por usuarios al body del documento anterior.

En este caso, como se cuenta con 2 documentos lo que se hará es crear 2 dataframes, uno con cada documento, para a continuación unirlos con la función `concat()` de pandas y ordenarlos según el id del Issue/PR para la clara visualización y mantener una estructura coherente entre cuerpo principal y comentarios asociados.

In [2]:
import pandas as pd

# Ruta de los archivos
path_gh_bodys = "../data/gh_comments/train-fine_tuning/gh_bodys_lastyear.csv"
path_gh_comments = "../data/gh_comments/train-fine_tuning/gh_comments_lastyear.csv"

# Carga de los archivos en DataFrames
df_bodys = pd.read_csv(path_gh_bodys)
df_comms = pd.read_csv(path_gh_comments)

In [3]:
print(df_bodys.columns)
print(df_comms.columns)

Index(['repo', 'is_pr', 'issue_number', 'comment_type', 'comment_id',
       'comment_created_at', 'comment_author', 'text', 'comment_url',
       'context_id', 'container_title', 'container_state', 'container_url',
       'container_created_at', 'container_updated_at', 'container_labels'],
      dtype='object')
Index(['kubernetes/kubernetes', 'False', '133680', 'issue_comment',
       'github_issuecomment_IC_kwDOAToIks6_4TOW', '2025-08-25T07:51:17Z',
       'k8s-ci-robot',
       'This issue is currently awaiting triage.\nIf a SIG or subproject determines this is a relevant issue, they will accept it by applying the triage/accepted label and provide further guidance.\nThe triage/accepted label can be added by org members by writing /triage accepted in a comment.\n\nInstructions for interacting with me using PR comments are available here.  If you have questions or suggestions related to my behavior, please file an issue against the kubernetes-sigs/prow repository.',
       'https://gi

Se ha cometido un error en la escritura de la cabecera de los comentarios por escribir siempre en el mismo documento y borrar su contenido en vez de eliminar el documento antes de comenzar con una nueva extracción. Vamos a tratar de repararlo sin tener que volver a realizar todo el proceso de extracción.

In [6]:
from pathlib import Path
path_gh_bodys = Path(path_gh_bodys)
path_gh_comments = Path(path_gh_comments)

EXPECTED_COLS = [
    'repo','is_pr','issue_number','comment_type','comment_id','comment_created_at','comment_author',
    'text','comment_url','context_id','container_title','container_state','container_url',
    'container_created_at','container_updated_at','container_labels'
]

def read_with_header_fix(p: Path) -> pd.DataFrame:
    # Se lee 1 fila para inspeccionar columnas
    probe = pd.read_csv(p, nrows=1)
    if list(probe.columns) == EXPECTED_COLS:
        return pd.read_csv(p)
    # Si no coincide, reinterpretamos: no hay cabecera -> header=None + names=EXPECTED_COLS
    return pd.read_csv(p, header=None, names=EXPECTED_COLS)

df_bodys = read_with_header_fix(path_gh_bodys)
df_comms = read_with_header_fix(path_gh_comments)

# Se unen ambos DataFrames
df_gh = pd.concat([df_bodys, df_comms], ignore_index=True)

# Tipos y ordenación
df_gh['comment_created_at'] = pd.to_datetime(df_gh['comment_created_at'], errors='coerce', utc=True)
df_gh.loc[df_gh['comment_created_at'].isna(), 'comment_created_at'] = pd.to_datetime(df_gh['container_created_at'], errors='coerce', utc=True)

order_map = {'issue_body':0, 'pr_body':0} # Bodies primero -> coherencia
df_gh['order'] = df_gh['comment_type'].map(order_map).fillna(1).astype(int)

df_gh = df_gh.sort_values(by=['repo','issue_number','order','comment_created_at','comment_id'], kind='mergesort').drop(columns=['order'])

# Normalizar booleano -> OPCIONAL
df_gh['is_pr'] = df_gh['is_pr'].astype(str).str.lower().map({'true':True, 'false':False})

In [13]:
# Muestra para comprobar que se ha ejecutado correctamente
print(df_bodys.columns)
print(df_comms.columns)

df_gh.head(10).T

Index(['repo', 'is_pr', 'issue_number', 'comment_type', 'comment_id',
       'comment_created_at', 'comment_author', 'text', 'comment_url',
       'context_id', 'container_title', 'container_state', 'container_url',
       'container_created_at', 'container_updated_at', 'container_labels'],
      dtype='object')
Index(['repo', 'is_pr', 'issue_number', 'comment_type', 'comment_id',
       'comment_created_at', 'comment_author', 'text', 'comment_url',
       'context_id', 'container_title', 'container_state', 'container_url',
       'container_created_at', 'container_updated_at', 'container_labels'],
      dtype='object')


,107919,107866,108041,108042,108038,107786,107867,108036,108037,107580
repo,electron-userland/electron-builder,electron-userland/electron-builder,electron-userland/electron-builder,electron-userland/electron-builder,electron-userland/electron-builder,electron-userland/electron-builder,electron-userland/electron-builder,electron-userland/electron-builder,electron-userland/electron-builder,electron-userland/electron-builder
is_pr,False,False,False,False,False,False,False,False,False,False
issue_number,690,2674,3009,3009,3124,3185,3322,3376,3376,3632
comment_type,issue_comment,issue_comment,issue_comment,issue_comment,issue_comment,issue_comment,issue_comment,issue_comment,issue_comment,issue_comment
comment_id,github_issuecomment_IC_kwDOAiVL48626ksv,github_issuecomment_IC_kwDOAiVL4866KmSG,github_issuecomment_IC_kwDOAiVL486w_HsW,github_issuecomment_IC_kwDOAiVL486x74HL,github_issuecomment_IC_kwDOAiVL486yE1aY,github_issuecomment_IC_kwDOAiVL4868Kgl1,github_issuecomment_IC_kwDOAiVL486577nr,github_issuecomment_IC_kwDOAiVL486d8ofG,github_issuecomment_IC_kwDOAiVL486yLFqm,github_issuecomment_IC_kwDOAiVL487ADWUN
comment_created_at,2025-07-14 10:14:03+00:00,2025-07-26 20:27:33+00:00,2025-06-13 07:10:06+00:00,2025-06-18 18:12:08+00:00,2025-06-19 10:41:44+00:00,2025-08-05 22:59:10+00:00,2025-07-25 17:00:17+00:00,2025-02-11 06:33:25+00:00,2025-06-19 22:14:59+00:00,2025-08-26 00:08:25+00:00
comment_author,Wiktor102,siikakamania,theIYD,prayash,minhtan143,oceangravity,Pritraj,hanzhenfang,devPablo,github-actions
text,Can confirm that specifying the appUrl option ...,Ok found problem by elimination. I have folder...,Anyone who was able to crack auto-update with ...,nice mna,This is a necessary feature,😭,"It is 2024, what happend on this issue? I hope...","same to me,","I'm facing the same issue, need dynamic ""url"" ...",This issue is stale because it has been open f...
comment_url,https://github.com/electron-userland/electron-...,https://github.com/electron-userland/electron-...,https://github.com/electron-userland/electron-...,https://github.com/electron-userland/electron-...,https://github.com/electron-userland/electron-...,https://github.com/electron-userland/electron-...,https://github.com/electron-userland/electron-...,https://github.com/electron-userland/electron-...,https://github.com/electron-userland/electron-...,https://github.com/electron-userland/electron-...
context_id,electron-userland/electron-builder#issue:690,electron-userland/electron-builder#issue:2674,electron-userland/electron-builder#issue:3009,electron-userland/electron-builder#issue:3009,electron-userland/electron-builder#issue:3124,electron-userland/electron-builder#issue:3185,electron-userland/electron-builder#issue:3322,electron-userland/electron-builder#issue:3376,electron-userland/electron-builder#issue:3376,electron-userland/electron-builder#issue:3632


Ahora sí están todos los comentarios bien ordenados. Antes de comenzar con el preprocesado vamos a guardar el dataframe en una base de datos.

In [16]:
import sqlite3
db_gh = "../data/gh_comments/train-fine_tuning/gh_dataset_lastyear.db"
con = sqlite3.connect(db_gh)
df_gh.to_sql('gh_comments', con, if_exists='replace', index=False)
con.close()

Ahora sí se procederá al procesamiento del dataset para dejarlo preparado para el modelo BERT que se utilizará, RoBERTa o DistilRoBERTa. Este proceso se va a definir en un script .py para su reutilización en otros puntos del proyecto (cuando se haga el de reddit, u otros comentarios de github), ejecutándolo en este punto del notebook.